In [5]:
using LinearAlgebra
using Random
using BenchmarkTools
using Statistics
using StaticArrays

In [1]:
using Polyester

In [2]:
Threads.nthreads()

4

In [2]:
function myloop7(;L = 100,theta = 10, alpha = 0.3, b = 1.2, K = 1, T = 10^3, H = 10)
    #Random.seed!(0)
    
    M=@SMatrix [1 0; b  0]

    strmtx = rand(0:1,L,L)
    paymtx = zeros(L,L)
    strselct = zeros(L,L)
    payselct =zeros(L,L)

    A_mtx =  zeros(L,L) #初始化压力矩阵A  
    A_self =  zeros(L,L)  #内心-伦理
    A_peer =  zeros(L,L)   #经济利润

    Fc = zeros(T+1)  #合作比例。有两个参数T+1, 1，则为 T+1 * 1 的列向量
    overstress = zeros(T+1)
    Fc[1] = sum(strmtx)/L^2

    for t = 1:T
        #4个邻居的策略
        strneigh = circshift(strmtx, 1) + circshift(strmtx, -1) + circshift(strmtx, (0, 1)) + circshift(strmtx, (0, -1))
        A_self +=  (strneigh .< 2.5)

        for i = 1:L
            for j = 1:L
                paymtx[i,j] =(@views [strmtx[i,j] 1-strmtx[i,j]] * M * [strneigh[i,j]; 4-strneigh[i,j]])[1]
            end
        end

          
        payave = (circshift(paymtx, 1)+circshift(paymtx, -1)+circshift(paymtx, (0, 1))+circshift(paymtx, (0, -1)))./4
       
        A_peer += (paymtx .< payave)

        A_mtx = (1-alpha).*A_self + alpha.*A_peer
        Agap = max.(theta .- A_mtx, 0)
        P1=exp.(-Agap)
 
        #randmtx = @SMatrix rand(1:4,L,L)   #每个个体选择一个邻居
        x = ((1,0), (-1,0), (0,1), (0,-1))

        for i =1:L
            for j = 1:L
                nei=rand(1:4,1)[1]
                i0=mod(i+x[nei][1]-1, L) +1
                j0=mod(j+x[nei][2]-1, L) +1
                strselct[i,j] = @views strmtx[i0, j0]
                payselct[i,j] = @views paymtx[i0, j0]
            end
        end
        K_A = Agap * K./theta
        P2 = 1 ./ (1 .+ exp.((paymtx .-payselct)./K_A))
        P2[isnan.(P2)] .= 0.5

        P1P2selectmtx = (P1.*P2 .> rand(L,L))

        strmtx[P1P2selectmtx.==1] .= strselct[P1P2selectmtx.==1]

        A_peer[P1P2selectmtx.==1] .= 0
        A_self[P1P2selectmtx.==1] .= 0
        Fc[t+1] =sum(strmtx)/L^2
        overstress[t+1]= sum(A_mtx .>= (theta+H)) /L^2
    end
    return Fc, overstress
end

myloop7 (generic function with 1 method)

In [3]:
@btime myloop7();

  2.676 s (40086020 allocations: 4.47 GiB)


In [4]:
function myloop8(;L = 100,theta = 10, alpha = 0.3, b = 1.2, K = 1, T = 10^3, H = 10)
    #Random.seed!(0)
    
    M=zeros(L,L,4)
    M[:,:,1] .=1
    M[:,:,3] .= b

    Strmtx = rand(0:1,L,L)

    A_mtx = zeros(L,L) #初始化压力矩阵A  
    A_peer = zeros(L,L)
    A_self = zeros(L,L)

    Fc = zeros(T+1,1)  #合作比例。有两个参数T+1, 1，则为 T+1 * 1 的列向量
    Fc[1] = sum(Strmtx)/L^2
    overstress = zeros(T+1,1)

    str1=zeros(L,L,4)  #自己
    str2=zeros(L,L,4)   #邻居

    Strsample = zeros(L,L,4)   #注意此处本来有off，但是下面4个层都没有off，乃取消  #off其实是payoff
    payoffsample = zeros(L,L,4)

    payoffsampleselct =zeros(L,L)
    Strsampleselct =zeros(L,L)

    for t = 1:T
        #4个邻居的策略
        Strsample[:,:,1]= circshift(Strmtx, 1)
        Strsample[:,:,2]= circshift(Strmtx, -1)  #下面的邻居
        Strsample[:,:,3]= circshift(Strmtx, (0, 1))  #右移一列-左边的邻居
        Strsample[:,:,4]= circshift(Strmtx, (0, -1))  #右边的邻居
        Strneighbor = dropdims(sum(Strsample,dims=3), dims=3)

        str1[:,:,1]=Strmtx
        str1[:,:,2]=Strmtx
        str1[:,:,3] = 1 .- Strmtx
        str1[:,:,4] = 1 .- Strmtx

        str2[:,:,1] = Strneighbor
        str2[:,:,2] = 4 .- Strneighbor
        str2[:,:,3] = Strneighbor
        str2[:,:,4] = 4 .- Strneighbor    

        payoffmtx = dropdims(sum(str1.*M.*str2,dims=3), dims=3)
        
        A_self +=  (Strneighbor .< 2.5)

        payoffsample[:,:,1]= circshift(payoffmtx, 1)  #行的变动-上下的邻居，首先是下移一行，即上面的邻居
        payoffsample[:,:,2]= circshift(payoffmtx, -1)  #下面的邻居
        payoffsample[:,:,3]= circshift(payoffmtx, (0, 1))  #右移一列-左边的邻居
        payoffsample[:,:,4]= circshift(payoffmtx, (0, -1))  #右边的邻居
       
        payoff_average = dropdims(sum(payoffsample,dims=3), dims=3)./4

        A_peer += (payoffmtx .< payoff_average)

        A_mtx = (1-alpha).*A_self + alpha.*A_peer
        Agap = max.(theta .- A_mtx, 0)
        P1=exp.(-Agap)
 
        randten= zeros(L,L,4)   #每个个体选择一个邻居
        randmtx = rand(1:4,L,L)
        for i =1:4
            randten[:,:,i] = (i.==randmtx)
        end
    
        payoffsampleselct = dropdims(sum(randten .* payoffsample, dims=3), dims=3)
        Strsampleselct = dropdims(sum(randten .* Strsample, dims=3), dims=3) 

        K_A = Agap * K./theta
        P2 = 1 ./ (1 .+ exp.((payoffmtx .-payoffsampleselct)./K_A))
        P2[isnan.(P2)] .= 0.5

        P1P2selectmtx = (P1.*P2 .> rand(L,L))

        Strmtx[P1P2selectmtx.==1] .= Strsampleselct[P1P2selectmtx.==1]

        A_peer[P1P2selectmtx.==1] .= 0
        A_self[P1P2selectmtx.==1] .= 0
        Fc[t+1] =sum(Strmtx)/L^2
        overstress[t+1]= sum(A_mtx .>= (theta+H)) /L^2
    end
    return Fc, overstress
end

myloop8 (generic function with 1 method)

In [5]:
@btime myloop8();

  1.458 s (162024 allocations: 3.60 GiB)


([0.5008; 0.5008; … ; 0.1969; 0.1981;;], [0.0; 0.0; … ; 0.0017; 0.0014;;])

In [6]:
function myloop6(;L = 100,theta = 10, alpha = 0.3, b = 1.2, K = 1, T = 10^3, H = 10)
    #Random.seed!(0)
    
    M=zeros(L,L,4)
    M[:,:,1] .=1
    M[:,:,3] .= b

    Strmtx = rand(0:1,L,L)

    A_mtx = zeros(L,L) #初始化压力矩阵A  
    A_peer = zeros(L,L)
    A_self = zeros(L,L)

    Fc = zeros(T+1,1)  #合作比例。有两个参数T+1, 1，则为 T+1 * 1 的列向量
    Fc[1] = sum(Strmtx)/L^2
    overstress = zeros(T+1,1)

    str1=zeros(L,L,4)  #自己
    str2=zeros(L,L,4)   #邻居

    Strsample = zeros(L,L,4)   #注意此处本来有off，但是下面4个层都没有off，乃取消
    payoffsample = zeros(L,L,4)

    payoffsampleselct =zeros(L,L)
    Strsampleselct =zeros(L,L)

    for t = 1:T
        #4个邻居的策略
        Strsample[:,:,1]= circshift(Strmtx, 1)
        Strsample[:,:,2]= circshift(Strmtx, -1)  #下面的邻居
        Strsample[:,:,3]= circshift(Strmtx, (0, 1))  #右移一列-左边的邻居
        Strsample[:,:,4]= circshift(Strmtx, (0, -1))  #右边的邻居
        Strneighbor = dropdims(sum(Strsample,dims=3), dims=3)

        str1[:,:,1]=Strmtx
        str1[:,:,2]=Strmtx
        str1[:,:,3] = 1 .- Strmtx
        str1[:,:,4] = 1 .- Strmtx

        str2[:,:,1] = Strneighbor
        str2[:,:,2] = 4 .- Strneighbor
        str2[:,:,3] = Strneighbor
        str2[:,:,4] = 4 .- Strneighbor    

        payoffmtx = dropdims(sum(str1.*M.*str2,dims=3), dims=3)
        
        A_self +=  (Strneighbor .< 2.5)

        payoffsample[:,:,1]= circshift(payoffmtx, 1)  #行的变动-上下的邻居，首先是下移一行，即上面的邻居
        payoffsample[:,:,2]= circshift(payoffmtx, -1)  #下面的邻居
        payoffsample[:,:,3]= circshift(payoffmtx, (0, 1))  #右移一列-左边的邻居
        payoffsample[:,:,4]= circshift(payoffmtx, (0, -1))  #右边的邻居
       
        payoff_average = dropdims(sum(payoffsample,dims=3), dims=3)./4

        A_peer += (payoffmtx .< payoff_average)

        A_mtx = (1-alpha).*A_self + alpha.*A_peer
        Agap = max.(theta .- A_mtx, 0)
        P1=exp.(-Agap)
 
        randmtx = rand(1:4,L,L)   #每个个体选择一个邻居

        for i =1:L
            for j = 1:L
                payoffsampleselct[i,j] = payoffsample[i,j,randmtx[i,j]]
                Strsampleselct[i,j] = Strsample[i,j,randmtx[i,j]]
            end
        end
        K_A = Agap * K./theta
        P2 = 1 ./ (1 .+ exp.((payoffmtx .-payoffsampleselct)./K_A))
        P2[isnan.(P2)] .= 0.5

        P1P2selectmtx = (P1.*P2 .> rand(L,L))

        Strmtx[P1P2selectmtx.==1] .= Strsampleselct[P1P2selectmtx.==1]

        A_peer[P1P2selectmtx.==1] .= 0
        A_self[P1P2selectmtx.==1] .= 0
        Fc[t+1] =sum(Strmtx)/L^2
        overstress[t+1]= sum(A_mtx .>= (theta+H)) /L^2
    end
    return Fc, overstress
end

myloop6 (generic function with 1 method)

In [7]:
@btime myloop6();

  1.162 s (124024 allocations: 2.53 GiB)


In [3]:
function myloop9(;L = 100,theta = 10, alpha = 0.3, b = 1.2, K = 1, T = 10^3, H = 10)
    #Random.seed!(0)
    
    M=zeros(L,L,4)
    M[:,:,1] .=1
    M[:,:,3] .= b

    Strmtx = rand(0:1,L,L)

    A_mtx = zeros(L,L) #初始化压力矩阵A  
    A_peer = zeros(L,L)
    A_self = zeros(L,L)

    Fc = zeros(T+1,1)  #合作比例。有两个参数T+1, 1，则为 T+1 * 1 的列向量
    Fc[1] = sum(Strmtx)/L^2
    overstress = zeros(T+1,1)

    str1=zeros(L,L,4)  #自己
    str2=zeros(L,L,4)   #邻居

    Strsample = zeros(L,L,4)   #注意此处本来有off，但是下面4个层都没有off，乃取消
    payoffsample = zeros(L,L,4)

    payoffsampleselct =zeros(L,L)
    Strsampleselct =zeros(L,L)

    @batch for t = 1:T
        #4个邻居的策略
        Strsample[:,:,1]= circshift(Strmtx, 1)
        Strsample[:,:,2]= circshift(Strmtx, -1)  #下面的邻居
        Strsample[:,:,3]= circshift(Strmtx, (0, 1))  #右移一列-左边的邻居
        Strsample[:,:,4]= circshift(Strmtx, (0, -1))  #右边的邻居
        Strneighbor = dropdims(sum(Strsample,dims=3), dims=3)

        str1[:,:,1]=Strmtx
        str1[:,:,2]=Strmtx
        str1[:,:,3] = 1 .- Strmtx
        str1[:,:,4] = 1 .- Strmtx

        str2[:,:,1] = Strneighbor
        str2[:,:,2] = 4 .- Strneighbor
        str2[:,:,3] = Strneighbor
        str2[:,:,4] = 4 .- Strneighbor    

        payoffmtx = dropdims(sum(str1.*M.*str2,dims=3), dims=3)
        
        A_self +=  (Strneighbor .< 2.5)

        payoffsample[:,:,1]= circshift(payoffmtx, 1)  #行的变动-上下的邻居，首先是下移一行，即上面的邻居
        payoffsample[:,:,2]= circshift(payoffmtx, -1)  #下面的邻居
        payoffsample[:,:,3]= circshift(payoffmtx, (0, 1))  #右移一列-左边的邻居
        payoffsample[:,:,4]= circshift(payoffmtx, (0, -1))  #右边的邻居
       
        payoff_average = dropdims(sum(payoffsample,dims=3), dims=3)./4

        A_peer += (payoffmtx .< payoff_average)

        A_mtx = (1-alpha).*A_self + alpha.*A_peer
        Agap = max.(theta .- A_mtx, 0)
        P1=exp.(-Agap)
 
        randmtx = rand(1:4,L,L)   #每个个体选择一个邻居

        for i =1:L
            for j = 1:L
                payoffsampleselct[i,j] = payoffsample[i,j,randmtx[i,j]]
                Strsampleselct[i,j] = Strsample[i,j,randmtx[i,j]]
            end
        end
        K_A = Agap * K./theta
        P2 = 1 ./ (1 .+ exp.((payoffmtx .-payoffsampleselct)./K_A))
        P2[isnan.(P2)] .= 0.5

        P1P2selectmtx = (P1.*P2 .> rand(L,L))

        Strmtx[P1P2selectmtx.==1] .= Strsampleselct[P1P2selectmtx.==1]

        A_peer[P1P2selectmtx.==1] .= 0
        A_self[P1P2selectmtx.==1] .= 0
        Fc[t+1] =sum(Strmtx)/L^2
        overstress[t+1]= sum(A_mtx .>= (theta+H)) /L^2
    end
    return Fc, overstress
end

myloop9 (generic function with 1 method)

In [6]:
@btime myloop9();

  695.422 ms (128024 allocations: 2.54 GiB)


In [1]:
rmse = 1.5; mse = 1.1; R2 = 0.94
println("Our model has a R^2 of $(R2), rmse of $(rmse), and mse of $(mse)")
println("Our model has a R^2 of $R2, rmse of $rmse, and mse of $mse")

Our model has a R^2 of 0.94, rmse of 1.5, and mse of 1.1
Our model has a R^2 of 0.94, rmse of 1.5, and mse of 1.1


In [3]:
println("Our model has a R^2 of ", R2)

Our model has a R^2 of 0.94
